# Classification problem with Fake and Real news
## BuffML www.buffml.com

In this project, I build a text classification to define whether or not a certain article is fake news or real news. Using Natural Language Processing methodologies in Python and Classification Theory, I reached an accuracy of 0.945455 for classifying news as fake.

In [ ]:
## This file has all imports and helper functions used throughout the notebook
%run python_helper.py
%matplotlib inline 

# Clean & Save Data

Inspecting the data files, we noticed several issues for processing the traning dataset correctly. Using Regular Expression, we convert all commas between quotations to a pipe, so the CSV parsing works correctly with all values in their correct columns.

In [ ]:
input_str = open("fake_or_real_news_training.csv", encoding= 'utf-8')

# Remove all new lines
noNewLines = re.sub("\n", "", input_str.read())
  
# re-add new line at end of each row
noNewLines = re.sub("X1,X2", "X1,X2\n", noNewLines)
  

noNewLines = re.sub(",FAKE[,]+", ",FAKE,,\n", noNewLines)
# noNewLines = re.sub(",FAKE,(?!,)",",FAKE,,\n",noNewLines)
# noNewLines = re.sub(",FAKE,,(?!,)",",FAKE,,\n",noNewLines)
  
noNewLines = re.sub(",REAL[,]+", ",REAL,,\n", noNewLines)
# noNewLines = re.sub(",REAL,(?!,)",",REAL,,\n",noNewLines)
# noNewLines = re.sub(",REAL,,(?!,)",",REAL,,\n",noNewLines)
  

# Replace any commas between two quotes with |
lines = noNewLines.split('\n')

def removeComma(g):
      t = g.groups()
      t = [t[0], t[1].replace(',', ' |'), t[2], t[3]]
      return "".join(t)

betweenQuotes = lambda line: re.sub(r'(.*,")(.*)(",)(.*)', lambda x: removeComma(x), line)

secondCol = lambda line: re.sub(r'^([0-9]+,)(.*,.*)(,\")(.*)$', lambda x: removeComma(x), line, 1)


lines = [betweenQuotes(l) for l in lines]
lines = [secondCol(l) for l in lines]

finalString = '\n'.join(lines)



### Save cleaned file

In [ ]:
file = open('fake_or_real_news_training_CLEANED.csv', 'w',encoding= 'utf-8')
file.write(finalString)
file.close()

# Data Preparation

In [ ]:
train = pd.read_csv("fake_or_real_news_training_CLEANED.csv")
test = pd.read_csv("fake_or_real_news_test.csv")

In [ ]:
len(train)

In [ ]:
len(test)

In [ ]:
train.head()

In [ ]:
train = train.drop(['X1', 'X2'], axis=1)

We study if the dataset is unbalanced. From the plot we see this is not the case, as there is a similar amount of Fake and Real news articles. No further actions have to be taken.

In [ ]:
from collections import Counter
ax = sns.countplot(train.label, order=[x for x, count in sorted(Counter(train.label).items(), key=lambda x: -x[1])])


for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/len(train)*100),
            ha="center") 
ax.set_title("Test dataset target")
show()

In [ ]:
test.head()

In order to not do double work by doing operations on our train and testset and to analyze general distributions of our data, we stack train and test in df.

In [ ]:
test['label'] = None  # empty label for test

df = pd.concat([train, test])

In [ ]:
len(df)

In [ ]:
df.tail()

# Data Preprocessing

In this part we will be cleaning the articles with the help of different NLP techniques, of which we will first explain the concept and its importance.

In order to take into account the title in our accuracy prediction, we created an extra column that combines text and title. We will not do separate predictions on the title since these might classify as e.g. Fake news, whether the actual text with more explanation tells a Real story.

In [ ]:
df['title_and_text'] = df['title'] +' '+ df['text']
df.tail()

preprocess() can be found in python_helper.py
Here you can read the explanations of the preprocess steps we took

1. lowercase the text

This preprocessing step is done so words van later be cross checked with the stopword and pos_tag dictionaries. For future analysis purposes, it could have been benefitial to analyze text with a lot of words in capital letters, by adding a flag variable.

2. remove the words counting just one letter

Idem step one.

3. remove the words that contain numbers

Idem step one.

4. tokenize the text and remove punctuation

We performed tokenization with the base python .string function, to split sentences into words (tokens). 

5. remove all stop words

Relevant analysis of the text depends on the most recurring words. Stopwords including words as "the", "as" and "and" appear a lot in a text, but do not give relevant explanation. For this reason they are removed.

6. remove tokens that are empty

After tokenization, we have to make sure all tokens taken into account contribute to the label prediction.

7. pos tag the text

We use the pos_tag function included in the ntlk library. This classifies our tokenized words as a noun, verb, adjective or adverb and adds to the understaning of the articles.

8. lemmatize the text

In order to normalize the text, we apply lemmatization. In this way, words with the same root are processed equally e.g. when took or taken are read in the text, they are lemmatized to take, infinitive of the two verbs.

In [ ]:
df['preprocessed_text'] = df['title_and_text'].apply(lambda x: preprocess(x))

In [ ]:
## Save preprocessed df
df.to_csv("fake_or_real_news_train_PREPROCESSED.csv", index=False)

In [ ]:
df = pd.read_csv("fake_or_real_news_train_PREPROCESSED.csv")
df = df.astype(object).replace(np.nan, 'None')

In [ ]:
df.tail()

### Split Train and Test again after pre-processing is done

In [ ]:
encoder, train, test, train_cv, train_holdout, train_cv_label, train_holdout_label = split_train_holdout_test(encoder, df)

In [ ]:
encoder

# Baseline Modelling

First, we create a dataframe called models to keep track of different models and their scores.

In [ ]:
models = pd.DataFrame(columns=['model_name', 'model_object', 'score'])

### Vectorizing dataset

For any text to be fed to a model, the text has to be transformed into numerical values. This process is called vectorizing and will be redone everytime a new feature is added.

In [ ]:
count_vect = CountVectorizer(analyzer = "word")

count_vectorizer = count_vect.fit(df.preprocessed_text)

train_cv_vector = count_vectorizer.transform(train_cv.preprocessed_text)
train_holdout_vector = count_vectorizer.transform(train_holdout.preprocessed_text)
test_vector = count_vectorizer.transform(test.preprocessed_text)

In [ ]:
count_vect.get_feature_names()[:10]

## Baseline Model 1: SVC

We create a baseline classification model with a support vector machine, a good model to handle complex classifications.

In [ ]:
SVC_classifier = runModel(encoder,
               train_cv_vector,
               train_cv_label,
               train_holdout_vector,
               train_holdout.label,
               "svc",
               "Baseline Model 1: SVC")
models.loc[len(models)] = SVC

## Baseline Model 2: Naïve Bayes

from IPython.display import Image
Image("/Users/Gerald/Personal Drive/IE MBD/Term III/Natural Language Processing/Ass 1/NLP Fake News Predection | Gerald | Hatem/real_vs_fake.png")

With this hand drawn example (text: *rude hell worth*), we explain why the Naïve Bayes model is helpful for our classification. The labels Real and Fake text are hidden, but every word, based on our training data, has a certain probability to belong to one of the two categories. The final score is calculated, multiplying all probabilities of the words (0.006 for real, 0.288 for fake). The algo thus does not take into account the order of the words in the multiplication. *rude hell worth* will be classified as fake.

In [ ]:
NB = runModel(encoder,
              train_cv_vector,
              train_cv_label,
              train_holdout_vector,
              train_holdout.label,
              "nb",
              "Baseline Model 2: Naiive Bayes")
models.loc[len(models)] = NB

## Baseline Model 3: MaxEnt Classifier

In [ ]:
maxEnt = runModel(encoder,
              train_cv_vector,
              train_cv_label,
              train_holdout_vector,
              train_holdout.label,
              "maxEnt",
              "Baseline Model 3: MaxEnt Classifier")
models.loc[len(models)] = maxEnt

# Baseline Models Summary

In [ ]:
models

# Feature Engin|eering

- Explicit POS tagging
- TF-IDF weighting
- Bigram Count Vectorizer

==> Select Final Model and predict on test

## 1. POS Tagging

Adding a prefix to each word with its type (Noun, Verb, Adjective,...).
e.g: I went to school => PRP-I VBD-went TO-to NN-school

Also, after lemmatization it will be 'VB-go NN-school', which indicates the semantics and distinguishes the purpose of the sentence.

This will help the classifier differentiate between different types of sentences.

In [ ]:
df['pos_tagged_text'] = df['preprocessed_text'].apply(lambda x: pos_tag_words(x))

In [ ]:
df.head()

### Rerun Models on pos-tagged text (FE1)

In [ ]:
encoder, train, test, train_cv, train_holdout, train_cv_label, train_holdout_label = split_train_holdout_test(encoder, df, False)

count_vect = CountVectorizer(analyzer = "word")

count_vectorizer = count_vect.fit(df.preprocessed_text)

train_cv_vector = count_vectorizer.transform(train_cv.pos_tagged_text)
train_holdout_vector = count_vectorizer.transform(train_holdout.pos_tagged_text)
test_vector = count_vectorizer.transform(test.pos_tagged_text)

a. SVC with FE1

In [ ]:
SVC_pos_tag = runModel(encoder,
               train_cv_vector,
               train_cv_label,
               train_holdout_vector,
               train_holdout.label,
               "svc",
               "SVC on pos-tagged text")
models.loc[len(models)] = SVC_pos_tag

b. NB_pos_tag with FE1

In [ ]:
NB_pos_tag = runModel(encoder,
              train_cv_vector,
              train_cv_label,
              train_holdout_vector,
              train_holdout.label,
              "nb",
              "Naiive Bayes on pos-tagged text")
models.loc[len(models)] = NB_pos_tag

c. maxEnt with FE1

In [ ]:
maxEnt_pos_tag = runModel(encoder,
              train_cv_vector,
              train_cv_label,
              train_holdout_vector,
              train_holdout.label,
              "maxEnt",
              "MaxEnt Classifier on pos-tagged text")
models.loc[len(models)] = maxEnt_pos_tag

In [ ]:
models

<h3 style="color:blue">
There seems to be a slight increase in Accuracy after pos-tagging.
</h3>

## 2. TF-IDF weighting

Try to add weight to each word using TF-IDF
<img src="https://cdn-images-1.medium.com/max/800/1*_OsV8gO2cjy9qcFhrtCdiw.jpeg" width="350px"/>

We are going to calculate the TFIDF score of each term in a piece of text. The text will be tokenized into sentences and each sentence is then considered a text item.

We will also apply those on the cleaned text and the concatinated POS_tagged text.

In [ ]:
df["clean_and_pos_tagged_text"] = df['preprocessed_text'] + ' ' + df['pos_tagged_text']

In [ ]:
df.head(1)

In [ ]:
encoder, train, test, train_cv, train_holdout, train_cv_label, train_holdout_label = split_train_holdout_test(encoder, df, False)

count_vect = CountVectorizer(analyzer = "word")

count_vectorizer = count_vect.fit(df.clean_and_pos_tagged_text)

train_cv_vector = count_vectorizer.transform(train_cv.clean_and_pos_tagged_text)
train_holdout_vector = count_vectorizer.transform(train_holdout.clean_and_pos_tagged_text)
test_vector = count_vectorizer.transform(test.clean_and_pos_tagged_text)


tf_idf = TfidfTransformer(norm="l2")
train_cv_tf_idf = tf_idf.fit_transform(train_cv_vector)
train_holdout_tf_idf = tf_idf.fit_transform(train_holdout_vector)
test_tf_idf = tf_idf.fit_transform(test_vector)  

### Rerun Models on preprocessed + pos-tagged (FE1) + TF-IDF weighted text (FE2)

a. SVC with FE1 and FE2

In [ ]:
SVC_tf_idf = runModel(encoder,
               train_cv_tf_idf,
               train_cv_label,
               train_holdout_tf_idf,
               train_holdout.label,
               "svc",
               "SVC on preprocessed+pos-tagged TF-IDF weighted text")
models.loc[len(models)] = SVC_tf_idf

b. NB with FE1 and FE2

In [ ]:
NB_tf_idf = runModel(encoder,
               train_cv_tf_idf,
               train_cv_label,
               train_holdout_tf_idf,
               train_holdout.label,
              "nb",
              "Naiive Bayes on preprocessed+pos-tagged TF-IDF weighted text")
models.loc[len(models)] = NB_tf_idf

c. maxEnt with FE1 and FE2

In [ ]:
maxEnt_tf_idf = runModel(encoder,
               train_cv_tf_idf,
               train_cv_label,
               train_holdout_tf_idf,
               train_holdout.label,
              "maxEnt",
              "MaxEnt on preprocessed+pos-tagged TF-IDF weighted text")
models.loc[len(models)] = maxEnt_tf_idf

In [ ]:
models

<h3 style="color:blue">
Using TF-IDF increased the score to ~94.5% with SVC and Max-Ent models.
<br><br>
Naive-Bayes rather decreased the score. Therefore we drop it from the pipeline.
</h3>

## 3. Use Bigram Vectorizer instead of regular vectorizer

For FE3, we use the Trigram vectorizer, which vectorizes **triplets of words** rather than each word separately. *In this short example sentence*, the trigrams are "In this short", "this short example" and "short example sentence".

In [ ]:
encoder, train, test, train_cv, train_holdout, train_cv_label, train_holdout_label = split_train_holdout_test(encoder, df, False)

trigram_vect = CountVectorizer(analyzer = "word", ngram_range=(1,2))

trigram_vect = count_vect.fit(df.clean_and_pos_tagged_text)

train_cv_vector = trigram_vect.transform(train_cv.clean_and_pos_tagged_text)
train_holdout_vector = trigram_vect.transform(train_holdout.clean_and_pos_tagged_text)
test_vector = trigram_vect.transform(test.clean_and_pos_tagged_text)

In [ ]:
tf_idf = TfidfTransformer(norm="l2")
train_cv_bigram_tf_idf = tf_idf.fit_transform(train_cv_vector)
train_holdout_bigram_tf_idf = tf_idf.fit_transform(train_holdout_vector)
test_bigram_tf_idf = tf_idf.fit_transform(test_vector)

### Rerun Models on preprocessed + pos-tagged (FE1) + TF-IDF weighted (FE2) + Trigram vectorized text (FE3)

a. SVC with FE1, FE2 and FE3

In [ ]:
SVC_trigram_tf_idf = runModel(encoder,
               train_cv_bigram_tf_idf,
               train_cv_label,
               train_holdout_bigram_tf_idf,
               train_holdout.label,
               "svc",
               "SVC on bigram vect.+ TF-IDF")
models.loc[len(models)] = SVC_trigram_tf_idf

b. maxEnt with FE1, FE2 and FE3

In [ ]:
encoder, train, test, train_cv, train_holdout, train_cv_label, train_holdout_label = split_train_holdout_test(encoder, df, False)

trigram_vect = CountVectorizer(analyzer = "word", ngram_range=(1,3))

trigram_vect = count_vect.fit(df.clean_and_pos_tagged_text)

train_cv_vector = trigram_vect.transform(train_cv.clean_and_pos_tagged_text)
train_holdout_vector = trigram_vect.transform(train_holdout.clean_and_pos_tagged_text)

In [ ]:
tf_idf = TfidfTransformer(norm="l2")
train_cv_trigram_tf_idf = tf_idf.fit_transform(train_cv_vector)
train_holdout_trigram_tf_idf = tf_idf.fit_transform(train_holdout_vector)

In [ ]:
maxEnt_tf_idf = runModel(encoder,
               train_cv_trigram_tf_idf,
               train_cv_label,
               train_holdout_trigram_tf_idf,
               train_holdout.label,
              "maxEnt",
              "MaxEnt on trigram vect.+ TF-IDF")
models.loc[len(models)] = maxEnt_tf_idf

In [ ]:
models

<h3 style="color:blue">
It looks like the "MaxEnt on trigram vect.+ TF-IDF" is the best model with the highest score. We will use it to predict and classify the testset.
</h3>

# Predicting on test dataset

## 1. Train on whole data and predict on test

### PREPROCESSED data

In [ ]:
test = pd.read_csv("fake_or_real_news_test.csv")
train = pd.read_csv("fake_or_real_news_training_CLEANED.csv")

In [ ]:
train['title_and_text'] = train['title'] +' '+ train['text']
train['preprocessed_text'] = train['title_and_text'].apply(lambda x: preprocess(x))

In [ ]:
test['title_and_text'] = test['title'] +' '+ test['text']
test['preprocessed_text'] = test['title_and_text'].apply(lambda x: preprocess(x))

In [ ]:
test.head()

In [ ]:
## Save preprocessed df
train.to_csv("fake_or_real_news_train_PREPROCESSED.csv", index=False)

In [ ]:
# Save preprocessed df
test.to_csv("fake_or_real_news_test_PREPROCESSED.csv", index=False)

In [ ]:
train = pd.read_csv("fake_or_real_news_train_PREPROCESSED.csv")
train = train.astype(object).replace(np.nan, 'None')

test = pd.read_csv("fake_or_real_news_test_PREPROCESSED.csv")
test = test.astype(object).replace(np.nan, 'None')

In [ ]:
test = test.astype(object).replace(np.nan, 'None')

In [ ]:
test.head()

In [ ]:
train.head()

### POS Tagging

In [ ]:
train['pos_tagged_text'] = train['preprocessed_text'].apply(lambda x: pos_tag_words(x))
test['pos_tagged_text'] = test['preprocessed_text'].apply(lambda x: pos_tag_words(x))

### merge clean and pos tagged

In [ ]:
train["clean_and_pos_tagged_text"] = train['preprocessed_text'] + ' ' + train['pos_tagged_text']
test["clean_and_pos_tagged_text"] = test['preprocessed_text'] + ' ' + train['pos_tagged_text']

In [ ]:
train.head(1)

In [ ]:
test.head(1)

## Modelling using MaxEnt on trigram vect.+ TF-IDF Grid Search Best params

### Trigram + Tfdif + classifier pipeline

In [ ]:
from sklearn.pipeline import Pipeline
trigram_vectorizer = CountVectorizer(analyzer = "word", ngram_range=(1,3))
tf_idf = TfidfTransformer(norm="l2")
classifier = LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

pipeline = Pipeline([
     ('trigram_vectorizer', trigram_vectorizer),
     ('tfidf', tf_idf),
     ('clf', classifier),
 ])


In [ ]:
pipeline.fit(train.clean_and_pos_tagged_text, encoder.fit_transform(train.label.values))

In [ ]:
import pickle
pickle.dump( pipeline, open( "pipeline.pkl", "wb" ) )

## 2. Predicting on test

In [ ]:
print(colored("Predicting on test", 'blue'))
test_predictions = test_predictions = pipeline.predict(test.clean_and_pos_tagged_text)


In [ ]:
test_predictions

In [ ]:
test_predictions_decoded = encoder.inverse_transform( test_predictions )

In [ ]:
predictions = test
predictions["label"] = test_predictions_decoded

In [ ]:
predictions.shape

In [ ]:
predictions.head()

In [ ]:
predictions.label.describe()

In [ ]:
import collections
ax = sns.countplot(predictions.label,
                order=[x for x, count in sorted(collections.Counter(predictions.label).items(),
                key=lambda x: -x[1])])


for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/len(predictions)*100),
            ha="center") 
ax.set_title("Test dataset target")
show()

In [ ]:
predictions.drop(columns=["title","text","title_and_text","preprocessed_text","pos_tagged_text","clean_and_pos_tagged_text"]).head()

In [ ]:
predictions.to_csv("TEST_PREDICTIONS.csv", index=False)